## Getting Chat Generated Articles from OpenAI

In [ ]:
#%pip install openai

In [ ]:
import openai
import pandas as pd
import numpy as np
import time

# Set api key here, left empty for account privacy reasons. You can add you own api key if you want to try
openai.api_key = ""

In [ ]:
# with open("C:\\Users\\graduate\\Box\\Courses taken\\CS 6830 Data Science in practice\\Final project\\openai_api.txt", 'r') as f:
#     api_key = f.read().strip()

# # Set api key here, left empty for account privacy reasons. You can add you own api key if you want to try
# openai.api_key = api_key

In [ ]:
# articles is a list of dictionaries. Each dictionary contains all the information needed to generate one news article.
# Information is gotten by scraping existing articles online
# Currently, the prompt requires the the new organization: 'source', wordcount the generated article should be: 'wordcount'
# and the headline: 'headline'. An example is shown below
articles = [{'source':'CNN', 'wordcount':'200', 'headline':'What is the Good Friday Agreement? How a historic deal ended the Troubles in Northern Ireland'}]

In [ ]:
def getFakeArticles(articles):
    fakeArticles = pd.DataFrame(columns=['Headline', 'Content'])
    for article in articles:
        prompt=[
            {"role": "system", "content": f"You are a journalist at {article['source']}"},
            {"role": "user", "content": f"Write an article with about {article['wordcount']} words with the headline '{article['headline']}'"},
        ]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=prompt
        )
        fakeArticles.loc[len(fakeArticles.index)] = [article['headline'], response['choices'][0]['message']['content']]
        print(f'Articles created: {len(fakeArticles)}')

        time.sleep(5)

    return fakeArticles

## Make CNN Fake Articles

In [ ]:
# df = pd.read_csv('CNN500_items.csv')

# ai_50 = df.sample(frac = 0.5)
# real_50 = df.drop(ai_50.index)

# articles = []
# for index, row in ai_50.iterrows():
#     articles.append({'source':'CNN','wordcount':len(row.Description.replace('\n','').split(' ')),'headline':row.Title.replace('\n','').strip()})

In [ ]:
# ai_gen = getFakeArticles(articles)

In [ ]:
# ai_gen.columns = ['Title', 'Content']
# ai_gen['Source'] = 1

# non_ai = real_50[['Title', 'Description']]
# non_ai.columns = ['Title', 'Content']
# non_ai['Source'] = 0

# cnn_mixed = pd.concat([ai_gen, non_ai], ignore_index=True)

In [ ]:
# cnn_mixed.to_csv(path_or_buf="cnn_mixed.csv")

## Make AP Fake Articles

In [ ]:
# df = pd.read_csv('AP501dataset.csv')

# df.loc[0].text
# ai_50 = df.sample(frac = 0.5)
# real_50 = df.drop(ai_50.index)

# articles = []
# for index, row in ai_50.iterrows():
#     articles.append({'source':'AP News','wordcount':len(row.text.replace('\n','').split(' ')),'headline':row.title.replace('\n','').strip()})

In [ ]:
# ai_gen = getFakeArticles(articles)

In [ ]:
# ai_gen.columns = ['Title', 'Content']
# ai_gen['Source'] = 1

# non_ai = real_50[['title', 'text']]
# non_ai.columns = ['Title', 'Content']
# non_ai['Source'] = 0

# ap_mixed = pd.concat([ai_gen, non_ai], ignore_index=True)

In [ ]:
# ap_mixed.to_csv("ap_mixed.csv")

## Make Fake Fox News Articles

In [ ]:
# df = pd.read_csv('fox260.csv')

# ai_50 = df.sample(frac = 0.5)
# real_50 = df.drop(ai_50.index)

# articles = []
# for index, row in ai_50.iterrows():
#     articles.append({'source':'Fox News','wordcount':len(row.text.replace('\n','').split(' ')),'headline':row.title.replace('\n','').strip()})

In [ ]:
# ai_gen = getFakeArticles(articles)

In [ ]:
# ai_gen.columns = ['Title', 'Content']
# ai_gen['Source'] = 1

# non_ai = real_50[['title', 'text']]
# non_ai.columns = ['Title', 'Content']
# non_ai['Source'] = 0

# fox_mixed = pd.concat([ai_gen, non_ai], ignore_index=True)

In [ ]:
# fox_mixed.to_csv("fox_mixed.csv")

## Megh's Analysis

#### Read dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read the three CSV files into separate DataFrames
ap_mixed1 = pd.read_csv('ap_mixed.csv')
display(ap_mixed1)

In [ ]:
cnn_mixed1 = pd.read_csv('cnn_mixed.csv')
display(cnn_mixed1)

In [ ]:
fox_mixed1 = pd.read_csv('fox_mixed.csv')
display(fox_mixed1)

#### merging 3 datasets into one

In [ ]:
# Concatenate the three DataFrames vertically (row-wise)
dataset = pd.concat([ap_mixed1, cnn_mixed1, fox_mixed1], ignore_index=True).reset_index()
dataset[750:1000]

In [ ]:
# # Optionally, you can drop duplicates if needed
# dataset.drop_duplicates(inplace=True)

dataset['Content'].replace('', np.nan, inplace=True)
dataset['Content'] = dataset.Content.apply(lambda x : x.strip())
dataset['Title'] = dataset.Title.apply(lambda x : x.strip())

dataset = dataset[dataset.Title != "Toxic smoke is spewing from an inferno at a recycling plant known as a 'fire hazard,' officials say. The flames could burn for days"].reset_index()


#drop NA values
dataset = dataset.dropna()

# Select only specific columns from the dataset
selected_columns = ['Title', 'Content', 'Source']
dataset_final = dataset[selected_columns]

display(dataset_final)
#final_dataset = dataset_final.to_csv('final_dataset.csv')
dataset_final.shape
# dataset_final.info


In [ ]:
import string
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

countplot of the real and fake news articles

In [ ]:
sns.countplot(data=dataset,x='Source')

convert the Content column in all lower case words

In [ ]:
##Convert 'Content' column to lowercase
dataset_final['L_Content'] = dataset_final['Content'].str.lower()
dataset_final.head()

remove stpwords and punctuation from the content of our news combination

In [ ]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuations=string.punctuation
dataset_final['L_Content'] = dataset_final.L_Content.apply(lambda x: ' '.join([word for word in x.split() 
                    if word not in (stop) and word[0] != '@' 
                    and word not in punctuations ]))
dataset_final.head()

remove punctuations

In [ ]:
dataset_final['L_Content'] = dataset_final['L_Content'].str.replace('[{}]'.format(string.punctuation), '')
dataset_final.head()

tokenize the news content

In [ ]:
# Load the NLTK tokenizer
nltk.download('punkt')

Tokenizing and Stemming

In [ ]:
from nltk.stem import PorterStemmer
# Create an instance of the PorterStemmer class
porter_stemmer = PorterStemmer()
# Replace null values with an empty string
dataset_final['L_Content'].fillna('', inplace=True)
# Tokenize the 'Content' column only for non-null values
dataset_final['L_Content'] = dataset_final['L_Content'].apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)


In [ ]:
# Define a function for stemming
def stem_words(tokens):
    return [porter_stemmer.stem(token) for token in tokens]

# Apply stemming to the 'Content' column
dataset_final['L_Content'] = dataset_final['L_Content'].apply(stem_words)

In [ ]:
dataset_final

## Naive Bayes Classification

#### Create a matrix with every unique word in it

In [ ]:
# Concatenate the list of words into a single string
dataset_final['L_Content'] = dataset_final['L_Content'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
# Convert the 'Content' column to lowercase
dataset_final['L_Content'] = dataset_final['L_Content'].str.lower()

In [ ]:
cv = CountVectorizer(ngram_range=(1,1), min_df=10)
vectorized_tweets = cv.fit_transform(dataset_final['L_Content'])

In [ ]:
# Split data into test and training dataset
X=vectorized_tweets
y=dataset_final['Source'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Naïve Bayes classification produces good results when we use it for textual data analysis such as Natural Language Processing
Gaussian NB: When we have continuous attribute values, we made an assumption that the values associated with each class are distributed according to Gaussian or Normal distribution
Multinominal NB: Multinomial Naïve Bayes algorithm is preferred to use on data that is multinomially distributed. It is one of the standard algorithms which is used in text categorization classification

In [ ]:
# Fit data to a multinomial Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
#display(y_pred)
p,r,f,s = precision_recall_fscore_support(y_test, y_pred)
print(p, r, f, s)

#### Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### Accuracy claculation

In [ ]:
# Training set accuracy
from sklearn.metrics import accuracy_score
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

### Check for overfitting and underfitting

In [ ]:
#print the scores on training and test set
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
array = [[tp, fp],[fn, tn]]
sns.heatmap(array, annot=True, annot_kws={"size": 16}, fmt='d', cmap='YlGnBu', vmin=0, vmax=140, cbar=True, linewidths=0.5, linecolor='k', square=True)
sns.despine(left=False, right=False, top=False, bottom=False)
plt.xlabel('Actual')
plt.ylabel('Predicted')
# plt.savefig('1x1.png')

In [ ]:
orig_article_sorted = clf.feature_log_prob_[0, :].argsort()[::-1]
ai_article_sorted = clf.feature_log_prob_[1, :].argsort()[::-1]
print('Orig article words:\n', np.take(cv.get_feature_names_out(), orig_article_sorted[:50]))
print('\nFake article words:\n', np.take(cv.get_feature_names_out(), ai_article_sorted[:50]))